In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import opendatasets as od
import tensorflow as tf
import cv2
from tqdm import *

In [2]:
dataset_directory = "C:\\Users\\amith\\Documents\\Datasets\\kaggle\\kaggle".replace("\\","/")
csv_directory = "C:\\Users\\amith\\Documents\\Datasets".replace("\\","/")

In [3]:
train_val_age = pd.read_csv(csv_directory + "/train_age.csv").sort_values(by="imageId")
train_val_gender = pd.read_csv(csv_directory + "/train_gender.csv").sort_values(by="imageId")

id = train_val_age["imageId"].values.tolist()
train_val_files = pd.DataFrame(columns=["imageId","Files"])
ctr = 0

for i in os.listdir(dataset_directory+"/train"):
    train_val_files.loc[ctr,"imageId"] = id[ctr]
    train_val_files.loc[ctr,"Files"] = i
    ctr += 1

train_val_files = train_val_files.sort_values(by="imageId")

train_val_output = pd.merge(left=train_val_age,right=train_val_gender,on="imageId",how="inner")
train_val_output = pd.merge(left=train_val_output,right=train_val_files,on="imageId",how="inner")

train_val_output = train_val_output.sort_values(by="imageId")

train_val_id = train_val_output["imageId"].values.tolist()
np.random.shuffle(train_val_id)


threshold = int(np.round(0.8 * len(train_val_id)))

train_id = train_val_id[0:threshold]
val_id = train_val_id[threshold:]

train_val_output.set_index(keys="imageId",drop=False,inplace=True)

train_output = train_val_output.loc[train_id,:]
val_output = train_val_output.loc[val_id,:]

train_output = train_output.reset_index(drop=True).drop(columns=["imageId"])
val_output = val_output.reset_index(drop=True).drop(columns=["imageId"])

In [4]:
train_output.head()

,age,gender,Files
0,60.0,1,008824.png
1,31.0,0,003467.png
2,32.0,0,008605.png
3,69.0,0,007564.png
4,81.0,0,006951.png


In [5]:
val_output.head()

,age,gender,Files
0,34.0,0,004337.png
1,25.0,0,000337.png
2,45.0,0,004975.png
3,44.0,0,001265.png
4,28.0,0,006441.png


In [6]:
train_images = []
train_data = {}
ctr = 0
for i in tqdm(iterable=train_output["Files"],desc="Processing train data"):
    ages = train_output.loc[ctr,"age"]
    genders = train_output.loc[ctr,"gender"]
    img_org = cv2.imread(dataset_directory + "/train/" + i)
    img_gray = cv2.cvtColor(src=img_org, code=cv2.COLOR_BGR2GRAY)
    img_short = cv2.resize(src=img_gray, dsize=(64,64))
    img = img_short / 255.0
    train_images.append(img)
    ctr += 1

train_images = np.array(train_images)

Processing train data: 100%|██████████| 8562/8562 [02:44<00:00, 51.96it/s]


In [7]:
dataset_directory + "/" + i

'C:/Users/amith/Documents/Datasets/kaggle/kaggle/003766.png'

In [8]:
dataset_directory + "/val/" + i

'C:/Users/amith/Documents/Datasets/kaggle/kaggle/val/003766.png'

In [9]:
val_images = []
val_data = {}
ctr = 0
for i in tqdm(iterable=val_output["Files"],desc="Processing val data"):
    ages = val_output.loc[ctr,"age"]
    genders = val_output.loc[ctr,"gender"]
    img_org = cv2.imread(dataset_directory + "/train/" + i)
    img_gray = cv2.cvtColor(src=img_org, code=cv2.COLOR_BGR2GRAY)
    img_short = cv2.resize(src=img_gray, dsize=(64,64))
    img = img_short / 255.0
    val_images.append(img)
    ctr += 1

val_images = np.array(val_images)

Processing val data: 100%|██████████| 2140/2140 [00:41<00:00, 52.14it/s]


In [10]:
train_images = train_images.reshape(tuple(list(train_images.shape) + [1]))
val_images = val_images.reshape(tuple(list(val_images.shape) + [1]))

In [11]:
val_images.shape

(2140, 64, 64, 1)

In [12]:
model_genders = tf.keras.models.Sequential(layers=[
    tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64,activation="relu"),
    tf.keras.layers.Dense(units=1,activation="softmax")
])

In [13]:
model_ages = tf.keras.models.Sequential(layers=[
    tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64,activation="relu"),
    tf.keras.layers.Dense(units=1,activation="relu")
])

In [14]:
train_ages = train_output["age"].values
train_genders = train_output["gender"].values

val_ages = val_output["age"].values
val_genders = val_output["gender"].values

In [18]:
model_genders.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.binary_crossentropy,metrics=['accuracy'])
model_ages.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.mean_absolute_error,metrics=['accuracy'])

In [ ]:
model_genders.fit(x=train_images,y=train_genders,batch_size=32,epochs=100,validation_data=(val_images,val_genders),verbose=1,shuffle=True,callbacks=tf.keras.callbacks.EarlyStopping(patience=2))

Epoch 1/100
268/268 [==============================] - 51s 186ms/step - loss: 0.5203 - accuracy: 0.4209 - val_loss: 0.3563 - val_accuracy: 0.4154
Epoch 2/100
268/268 [==============================] - 50s 186ms/step - loss: 0.3622 - accuracy: 0.4209 - val_loss: 0.3038 - val_accuracy: 0.4154
Epoch 3/100
268/268 [==============================] - 51s 191ms/step - loss: 0.2996 - accuracy: 0.4209 - val_loss: 0.2633 - val_accuracy: 0.4154
Epoch 4/100
268/268 [==============================] - 61s 229ms/step - loss: 0.2613 - accuracy: 0.4209 - val_loss: 0.2499 - val_accuracy: 0.4154
Epoch 5/100
268/268 [==============================] - 63s 235ms/step - loss: 0.2223 - accuracy: 0.4209 - val_loss: 0.2409 - val_accuracy: 0.4154
Epoch 6/100
268/268 [==============================] - 62s 233ms/step - loss: 0.1871 - accuracy: 0.4209 - val_loss: 0.2587 - val_accuracy: 0.4154
Epoch 7/100
268/268 [==============================] - 69s 256ms/step - loss: 0.1452 - accuracy: 0.4209 - val_loss: 0.2342 -

In [17]:
model_ages.fit(x=train_images,y=train_ages,batch_size=32,epochs=100,validation_data=(val_images,val_ages),verbose=1,shuffle=True,callbacks=tf.keras.callbacks.EarlyStopping(patience=2))

Epoch 1/100
268/268 [==============================] - 44s 160ms/step - loss: 16.0823 - accuracy: 0.0000e+00 - val_loss: 12.3604 - val_accuracy: 0.0000e+00
Epoch 2/100
222/268 [=======================>......] - ETA: 10s - loss: 12.1099 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [33]:
tf.keras.models.save_model(model=model_genders,filepath="C:/Users/amith/Documents/Datasets/model_genders.h5",overwrite=True)
tf.keras.models.save_model(model=model_ages,filepath="C:/Users/amith/Documents/Datasets/model_ages.h5",overwrite=True)

C:\Users\amith\AppData\Local\Temp\ipykernel_20364\2445089383.py:1: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model=model_1,filepath="C:/Users/amith/Documents/Datasets/model_1.h5",overwrite=True)
